In [1]:
import asyncio
import solana
import json
from solana.rpc.async_api import AsyncClient
from solana.exceptions import SolanaRpcException, SolanaExceptionBase
import nest_asyncio
#use logger in script instead of print

In [2]:
nest_asyncio.apply() # This is needed to run the async loop in Jupyter

In [3]:
mainnet_url = "https://api.mainnet-beta.solana.com"

In [4]:
def check_transaction_program(transaction) -> bool:
    """
    Verify that the first and last transaction programs are spl-token
    """

    first_transaction_program = None
    last_transaction_program = None
    
    try:
        if "program" in transaction["meta"]["innerInstructions"][0]["instructions"][0].keys():
            first_transaction_program = transaction["meta"]["innerInstructions"][0]["instructions"][0]["program"]
            last_transaction_program = transaction["meta"]["innerInstructions"][0]["instructions"][-1]["program"]
        elif "program" in transaction["meta"]["innerInstructions"][0]["instructions"][1].keys():
            first_transaction_program = transaction["meta"]["innerInstructions"][0]["instructions"][1]["program"]
            last_transaction_program = transaction["meta"]["innerInstructions"][0]["instructions"][-1]["program"]
    except:
        pass
        
    if first_transaction_program != "spl-token":
        return False
    
    if last_transaction_program != "spl-token":
        return False
        
    return True

def check_transaction_type(transaction) -> bool:
    """
    Verify that the first and last transaction types are transfer

    """
    try:
        if "parsed" in transaction["meta"]["innerInstructions"][0]["instructions"][0].keys():
            first_transaction_type = transaction["meta"]["innerInstructions"][0]["instructions"][0]["parsed"]["type"]
            second_transaction_type = transaction["meta"]["innerInstructions"][0]["instructions"][-1]["parsed"]["type"]
        elif "parsed" in transaction["meta"]["innerInstructions"][0]["instructions"][1].keys():
            first_transaction_type = transaction["meta"]["innerInstructions"][0]["instructions"][1]["parsed"]["type"]
            second_transaction_type = transaction["meta"]["innerInstructions"][0]["instructions"][-1]["parsed"]["type"]
    except:
        first_transaction_type = None
        second_transaction_type = None

    if first_transaction_type != "transfer":
        return False
    
    if second_transaction_type != "transfer":
        return False
    return True

def check_token_address(transaction) -> bool:
    """
    Verify that the token address source is USDC and the token address destination is wrapped solana
    
    """
    usdc_token_address = "EPjFWdd5AufqSSqeM2qN1xzybapC8G4wEGGkZwyTDt1v"
    wrapped_btc = "9n4nbM75f5Ui33ZbPYXn59EwSgE8CGsHtAeTH5YFeJ9E"
    wrapped_solana = "So11111111111111111111111111111111111111112"
    
    token_address_source = None
    token_address_destination = None

    try:
        token_address_source = transaction["meta"]["preTokenBalances"][0]["mint"]
        token_address_destination = transaction["meta"]["preTokenBalances"][-1]["mint"]
    except:
        pass
    
    if token_address_source not in [usdc_token_address, wrapped_btc]:
        return False
    
    if token_address_destination != wrapped_solana:
        return False
        
    return True

def check_error_status(transaction) -> bool:
    """
    Verify that the error and status are None
    
    """
    error, status = "", ""
    try:
        error = transaction["meta"]["err"]
        status = transaction["meta"]["status"]["Ok"]
    except:
        pass
   
    if (error is None) and (status is None):
        return True

    return False

In [5]:
async def fetch_block(url):
    async with AsyncClient(url) as client:
        latest_blockhash = await client.get_latest_blockhash()
        slot = json.loads((latest_blockhash).to_json())["result"]["context"]["slot"]
    return slot # True
        


In [6]:
async def stream_data(url):
    while True:
        data = await fetch_block(url)
        if data is not None:
            return data  # Process or handle the data as needed

    await asyncio.sleep(1)


In [7]:
def retrieve_transaction_details(transaction):
    if check_token_address(transaction) and check_transaction_program(transaction) and check_transaction_type(transaction) and check_error_status(transaction):
        if len(transaction["meta"]["innerInstructions"]) > 0:
            inner_instructions = transaction["meta"]["innerInstructions"]
            if "parsed" in inner_instructions[0]["instructions"][0].keys() and "parsed" in inner_instructions[0]["instructions"][-1].keys():     
                try:
                    first_instruction = inner_instructions[0]["instructions"][0]["parsed"]["info"]
                    last_instruction = inner_instructions[0]["instructions"][-1]["parsed"]["info"]
                except:
                    first_instruction = inner_instructions[0]["instructions"][1]["parsed"]["info"]
                    last_instruction = inner_instructions[0]["instructions"][-1]["parsed"]["info"]

                if ("amount" in first_instruction.keys()) and ("amount" in last_instruction.keys()):
                    source_amount = first_instruction["amount"]
                    destination_amount = last_instruction["amount"]
                    source_account = first_instruction["source"]
                    destination_account = last_instruction["destination"]
                    transaction_fees = transaction["meta"]["fee"]
                    signature = transaction["transaction"]["signatures"][0]

                    send = {
                        "source_amount": source_amount, "destination_amount": destination_amount, 
                        "source_account": source_account, "destination_account": destination_account, 
                        "transaction_fees": transaction_fees, "signature": signature
                        }
            

                    return send

In [8]:
async def retrieve_block_transactions(url):
    while True:
        try:
            async with AsyncClient(url) as client:
                block_number = await stream_data(url)
                blk_transaction = await client.get_block(block_number, "jsonParsed", max_supported_transaction_version=0)
                return blk_transaction
        except:
            pass

In [9]:
async def main():
    while True:
        transactions = await retrieve_block_transactions(mainnet_url)
        transactions = json.loads(transactions.to_json())
        for transaction in transactions["result"]["transactions"]:
            data = retrieve_transaction_details(transaction)
            if data is not None:
                data["block_time"] = transactions["result"]["blockTime"]
                data["block_number"] = transactions["result"]["parentSlot"] + 1 # block number is parent slot + 1
                print(data)
            

In [10]:
asyncio.run(main())

{'source_amount': '30921856', 'destination_amount': '30924839', 'source_account': '1B6Ua3MNkUZLCJ7zJpdd46i8SMCrbQpw13kG74oFKbp', 'destination_account': '1B6Ua3MNkUZLCJ7zJpdd46i8SMCrbQpw13kG74oFKbp', 'transaction_fees': 5340, 'signature': '4U6njqdsNHHE1hEeQa2yFEPzJ3ZvToFKSn1zcS8n8obpMyAGWmE4aWuPiXaUTosrASQaXvFY59PnfAjbdMDwUS2t', 'block_time': 1707526191, 'block_number': 247181401}
